In [ ]:
### FEATURE CALCULATION
### May 23, 2025
### Lorena Lopez Dominguez
### From https://brainconn.readthedocs.io/en/latest/api.html
### kernel: base (Python 3.12.8)

In [3]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import bct

In [4]:
# Set working directory
base_dir = '/Users/Lore/Desktop/brainhack/project/BrainHack-Project-'
data_dir = os.path.join(base_dir, 'FCData_18')
os.chdir(base_dir)

# Create results directory if it doesn't exist
results_dir = os.path.join(base_dir, 'results')
os.makedirs(results_dir, exist_ok=True)

In [5]:
# Load community index vector
civ = pd.read_table('network_roi_7networks.txt')

In [6]:
# Initialize DataFrames to store all subjects' data
all_zscores = pd.DataFrame()
all_partcoef = pd.DataFrame()
all_strength = pd.DataFrame()

In [7]:
# Initialize accumulators for average plots
sum_adj_wei = None
sum_adj_bin = None
subject_count = 0

In [8]:
# Loop through files in directory
for filename in os.listdir(data_dir):
    if filename.startswith('data_sub-') and filename.endswith('.txt'):
        subject_name = re.search(r'data_sub-(.*)\.txt', filename).group(1)
        file_path = os.path.join(data_dir, filename)

        # Load FC data
        FCData = np.loadtxt(file_path, delimiter=',', skiprows=1)

        # Transform to weighted and binary adjacency matrices
        adj_wei = FCData - np.eye(FCData.shape[0])
        adj_bin = bct.utils.binarize(bct.utils.threshold_proportional(adj_wei, 0.2))
        
        # Update accumulators
        if sum_adj_wei is None:
             sum_adj_wei = adj_wei.copy()
             sum_adj_bin = adj_bin.copy()
        else:
             sum_adj_wei += adj_wei
             sum_adj_bin += adj_bin
             
        subject_count += 1

        # Calculate centrality measures
        mod_z = bct.centrality.module_degree_zscore(adj_wei, civ.network_group)
        part_coef = bct.centrality.participation_coef(adj_wei, civ.network_group)
        strength = bct.degree.strengths_und(adj_wei)

        # Store in DataFrames with subject name as column
        all_zscores[subject_name] = mod_z
        all_partcoef[subject_name] = part_coef
        all_strength[subject_name] = strength

        # # Optional: Save plots per subject (can comment out if not needed)
        # fig, ax = plt.subplots(figsize=(7, 7))
        # ax.imshow(adj_wei, cmap='viridis')
        # plt.title(f'Weighted adjacency matrix: {subject_name}')
        # fig.colorbar(ax.images[0])
        # plt.savefig(os.path.join(data_dir, f'{subject_name}_plot_wei.png'))
        # plt.close()

        # fig, ax = plt.subplots(figsize=(7, 7))
        # ax.imshow(adj_bin, cmap='viridis')
        # plt.title(f'Binary adjacency matrix: {subject_name}')
        # fig.colorbar(ax.images[0])
        # plt.savefig(os.path.join(data_dir, f'{subject_name}_plot_bin.png'))
        # plt.close()

In [9]:
# Transpose to have subjects as rows
z_df = all_zscores.T.reset_index()
pc_df = all_partcoef.T.reset_index()
strength_df = all_strength.T.reset_index()

# Rename the index column to 'ID'
z_df.rename(columns={'index': 'ID'}, inplace=True)
pc_df.rename(columns={'index': 'ID'}, inplace=True)
strength_df.rename(columns={'index': 'ID'}, inplace=True)

z_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(z_df.shape[1] - 1)]
pc_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(pc_df.shape[1] - 1)]
strength_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(strength_df.shape[1] - 1)]


In [12]:
# Save all features to CSV
z_df.to_csv(os.path.join(results_dir, 'within_module_z_all_adhd.csv'), index=False)
pc_df.to_csv(os.path.join(results_dir, 'participation_coeff_all_adhd.csv'), index=False)
strength_df.to_csv(os.path.join(results_dir, 'node_strength_all_adhd.csv'), index=False)


In [13]:
# Compute average matrices
avg_adj_wei = sum_adj_wei / subject_count
avg_adj_bin = sum_adj_bin / subject_count

# Plot average weighted adjacency matrix
fig, ax = plt.subplots(figsize=(7, 7))
plot_wei_avg = ax.imshow(avg_adj_wei, cmap='viridis')
plt.title('Average Weighted Adjacency Matrix')
fig.colorbar(plot_wei_avg)
plt.savefig(os.path.join(results_dir, 'average_weighted_adjacency_adhd.png'))
plt.close()

# Plot average binary adjacency matrix
fig, ax = plt.subplots(figsize=(7, 7))
plot_bin_avg = ax.imshow(avg_adj_bin, cmap='viridis')
plt.title('Average Binary Adjacency Matrix')
fig.colorbar(plot_bin_avg)
plt.savefig(os.path.join(results_dir, 'average_binary_adjacency_adhd.png'))
plt.close()